# Output parsers

https://python.langchain.com/docs/how_to/output_parser_structured/

In [1]:
from langchain_ollama import ChatOllama

DEEP_THINKING_INSTRUCTION = "Enable deep thinking subroutine.\n\n"

In [2]:
model = ChatOllama(model="cogito:8b", num_gpu=256)

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field, model_validator

In [4]:
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    @model_validator(mode="before")
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get("setup")
        if setup and setup[-1] != "?":
            raise ValueError("Badly formed question!")
        return values

In [5]:
parser = PydanticOutputParser(pydantic_object=Joke)

In [8]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [9]:
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})
parser.invoke(output)

Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')

In [10]:
output

AIMessage(content='{\n    "setup": "Why did the chicken cross the road?",\n    "punchline": "To get to the other side!"\n}', additional_kwargs={}, response_metadata={'model': 'cogito:8b', 'created_at': '2025-05-03T15:00:11.2521054Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1162762900, 'load_duration': 26024300, 'prompt_eval_count': 209, 'prompt_eval_duration': 215991800, 'eval_count': 30, 'eval_duration': 920225500, 'model_name': 'cogito:8b'}, id='run-2a262d47-e1b2-424c-9421-6a165d0b55f6-0', usage_metadata={'input_tokens': 209, 'output_tokens': 30, 'total_tokens': 239})

In [11]:
chain = prompt | model | parser
chain.invoke({"query": "Teach me something as question."})

Joke(setup='What did one neuron say to the other?', punchline='Nothing, they just passed along a signal.')

In [12]:
list(chain.stream({"query": "Tech me something weird a lot!"}))

[Joke(setup="Why did the tech company's CEO only own one pair of shoes?", punchline=''),
 Joke(setup="Why did the tech company's CEO only own one pair of shoes?", punchline='Because'),
 Joke(setup="Why did the tech company's CEO only own one pair of shoes?", punchline='Because every'),
 Joke(setup="Why did the tech company's CEO only own one pair of shoes?", punchline='Because every time'),
 Joke(setup="Why did the tech company's CEO only own one pair of shoes?", punchline='Because every time they'),
 Joke(setup="Why did the tech company's CEO only own one pair of shoes?", punchline='Because every time they thought'),
 Joke(setup="Why did the tech company's CEO only own one pair of shoes?", punchline='Because every time they thought they'),
 Joke(setup="Why did the tech company's CEO only own one pair of shoes?", punchline='Because every time they thought they needed'),
 Joke(setup="Why did the tech company's CEO only own one pair of shoes?", punchline='Because every time they thought 